In [1]:
# load json file
import json
import os

# load json file
def load_json_file(file_path):
    """
    Load a JSON file and return its contents.
    
    Args:
        file_path (str): The path to the JSON file.
        
    Returns:
        dict: The contents of the JSON file.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    return data

In [2]:
childlens_annotations = load_json_file('/home/nele_pauline_suffo/ProcessedData/childlens_annotations/100898.MP4.json')

In [11]:
childlens_annotations['metadata']

{'id': 91342905,
 'name': '100898.MP4',
 'duration': 1023522500,
 'width': 2304,
 'height': 1296,
 'lastAction': {'timestamp': 1735907353856,
  'email': 'sa.partner.ana+adm1@gmail.com'},
 'projectId': 188494,
 'url': 'https://sa-customer-facing.s3.amazonaws.com/client-data/max-planck/100898.MP4',
 'status': 'Completed',
 'error': None,
 'annotatorEmail': 'sa.partner.ana+1@gmail.com',
 'qaEmail': None,
 '__params__': {'team_id': 22691,
  'project_id': 188494,
  'asset_id': 91342905,
  'folder_id': 534882,
  'timestamp': 1726165548863},
 'etag': 'caf980937159789bd08116fd191ee959'}

In [17]:
childlens_annotations['instances'][1]['parameters'][0]

{'start': 0,
 'end': 1825263,
 'timestamps': [{'attributes': [{'id': 6169960,
     'groupId': 975662,
     'name': 'Child Talking',
     'groupName': 'Type of Action'},
    {'id': 6169969, 'groupId': 975663, 'name': 'No', 'groupName': 'Alone?'},
    {'id': 6169973,
     'groupId': 975664,
     'name': 'Adult',
     'groupName': '1st Person Age Group'},
    {'id': 6169976,
     'groupId': 975665,
     'name': 'Female',
     'groupName': '1st Person Gender'}],
   'timestamp': 0},
  {'attributes': [{'id': 6169960,
     'groupId': 975662,
     'name': 'Child Talking',
     'groupName': 'Type of Action'},
    {'id': 6169969, 'groupId': 975663, 'name': 'No', 'groupName': 'Alone?'},
    {'id': 6169973,
     'groupId': 975664,
     'name': 'Adult',
     'groupName': '1st Person Age Group'},
    {'id': 6169976,
     'groupId': 975665,
     'name': 'Female',
     'groupName': '1st Person Gender'}],
   'timestamp': 1825263}]}

In [ ]:
import json

# Load your annotation JSON file
with open("/home/nele_pauline_suffo/ProcessedData/childlens_annotations/100898.MP4.json", "r") as f:
    annotations = json.load(f)

# Define the set of relevant labels to keep
valid_action_names = {"Child Talking", "Other Person Talking", "Overheard Speech"}

# The URI (e.g., filename without extension) you want to include in RTTM
uri = childlens_annotations['metadata']['name']

rttm_lines = []

# Loop through all annotation instances
for instance in annotations['instances']:
    try:
        timestamps = instance["parameters"][0]["timestamps"]
        times = instance["parameters"][0]
        for ts in timestamps:
            action_type = next(
                (attr["name"] for attr in ts["attributes"]
                 if attr["groupName"] == "Type of Action"),
                None
            )
            if action_type in valid_action_names:
                start = times["start"] / 1000000  # Convert microseconds to seconds
                # Get end from the next timestamp or from the event end
                end = times["end"] / 1000000
                duration = end - start
                speaker_id = action_type.replace(" ", "_").lower()

                rttm_line = f"SPEAKER {uri} 1 {start:.3f} {duration:.3f} <NA> <NA> {speaker_id} <NA> <NA>"
                rttm_lines.append(rttm_line)
                break  # Only use the first matching timestamp block per instance
    except Exception as e:
        print(f"Skipping instance due to error: {e}")

# Save RTTM lines to file
with open("output.rttm", "w") as f:
    for line in rttm_lines:
        f.write(line + "\n")